In [1]:
import hashlib

In [8]:
def sha256(input_string):
   return hashlib.sha256(input_string.encode()).hexdigest()


In [10]:
sha256("I") == 'a83dd0ccbffe39d071cc317ddf6e97f5c6b1c87af91919271f9fa140b0508c6c'

True

In [12]:
sha256('love') =='686f746a95b6f836d7d70567c302c3f9ebb5ee0def3d1220ee9d4e9f34f5e131'

True

In [46]:
users = {
    'admin':'8c6976e5b5410415bde908bd4dee15dfb167a9c873fc4bb8a81f6f2ab448a918', #sha256('admin')
    'user':'2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824', #sha256('hello')
}
def authenticate(username,password):
    if username in users and users[username] == sha256(password):
        return True
    return False

In [50]:
authenticate('user','helo')

False

In [23]:
chars = 'abcdefghijklmnopqrstuvxyz'

def generate_combinations(length):
    if length == 0:
        return ['']
    smaller_combinations = generate_combinations(length - 1)
    return [s + char for s in smaller_combinations for char in chars]

def hack_sha256_fixed_size(hash_value, password_length):
    for password in generate_combinations(password_length):
        if sha256(password) == hash_value:
            return password
    
    return None



In [24]:
hack_sha256_fixed_size('8c6976e5b5410415bde908bd4dee15dfb167a9c873fc4bb8a81f6f2ab448a918',5)

'admin'

In [25]:
hack_sha256_fixed_size('2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824',5)

'hello'

In [26]:
hack_sha256_fixed_size('a819d7cd38e9101be2e496298e8bf426ce9cdf78d2af35ddf44c6ad25d50158b',5)

'crypt'

In [27]:
hack_sha256_fixed_size('688787d8ff144c502c7f5cffaafe2cc588d86079f9de88304c26b0cb99ce91c6',3)

'asd'

In [28]:
hack_sha256_fixed_size('7ec658e98073955c48314d0146593497a163d79f4e1dfea4bab03b79af227214',4)

'elte'

In [29]:
def hack_sha256(hash_value):
    for length in range(1, 11):
        result = hack_sha256_fixed_size(hash_value, length)
        if result is not None:
            return result
    return None

In [30]:
hack_sha256('8c6976e5b5410415bde908bd4dee15dfb167a9c873fc4bb8a81f6f2ab448a918')

'admin'

In [31]:
hack_sha256('2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824')

'hello'

In [32]:
hack_sha256('a819d7cd38e9101be2e496298e8bf426ce9cdf78d2af35ddf44c6ad25d50158b')

'crypt'

In [33]:
hack_sha256('688787d8ff144c502c7f5cffaafe2cc588d86079f9de88304c26b0cb99ce91c6')

'asd'

In [34]:
hack_sha256('7ec658e98073955c48314d0146593497a163d79f4e1dfea4bab03b79af227214')

'elte'

In [39]:
def authenticate_with_pepper(username, password):
    pepper_prefix = 'this_can_help_to_confuse_the_attacker_'
    users_with_pepper = {
        'admin': {'passwordHash': '89e6b5ed137e3864d99ec9b421cf6f565d611f4c2b98e31a7d353d63aa748e9c'},
        'user': {'passwordHash': '6dc765830e675d5fa4a9afb248be09a0407f6353d44652fd9b36038884a76323'}
    }

    password_hash = sha256(pepper_prefix + password)

    return users_with_pepper.get(username, {}).get('passwordHash') == password_hash

In [40]:
authenticate_with_pepper('admin', 'admin')

True

In [41]:
authenticate_with_pepper('admin', 'admin2')

False

In [42]:
authenticate_with_pepper('user', 'hello')

True

In [44]:
def authenticate_with_pepper_and_salt(username, password):
    pepper_prefix = 'this_can_help_to_confuse_the_attacker_'
    users_with_pepper_and_salt = {
        'admin': {'passwordHash': 'd3eab7f4d6974f1db32b9cd9923fce9b434b28dc229b6582b845f1fca770d9f7', 'salt': "5294976873732394418"},
        'user': {'passwordHash': '976c73e0b408c89df3c1a12c3b0c45a6fee71bc1de5b47a88fae1a5e69ba6e28', 'salt': '1103733363818826232'}
    }

    user_data = users_with_pepper_and_salt.get(username)

    if not user_data:
        return False

    password_hash = sha256(pepper_prefix + password + user_data['salt'])

    return password_hash == user_data['passwordHash']




In [45]:
test_results_salt_pepper = {
    ('admin', 'admin'): authenticate_with_pepper_and_salt('admin', 'admin'),
    ('admin', 'admin2'): authenticate_with_pepper_and_salt('admin', 'admin2'),
    ('user', 'hello'): authenticate_with_pepper_and_salt('user', 'hello'),
    ('user', 'helo'): authenticate_with_pepper_and_salt('user', 'helo')
}
test_results_salt_pepper

{('admin', 'admin'): True,
 ('admin', 'admin2'): False,
 ('user', 'hello'): True,
 ('user', 'helo'): False}

# Rainbow attack

In [24]:
import hashlib

def read_passwords_from_file(file_path):
    password_list = []
    with open(file_path, 'r') as file:
        for line in file:
            password_list.append(line.strip())
    return password_list

def create_rainbow_table():
    rainbow_table = {}
    
    password_list = read_passwords_from_file('./milw0rm-dictionary.txt')

    for password in password_list:
        hashed = hashlib.sha256(password.encode()).hexdigest()
        rainbow_table[hashed] = password
    return rainbow_table
def search_rainbow_table(rainbow_table, hash_to_find):
    return rainbow_table.get(hash_to_find, None)


In [25]:
import string
import random

chars = string.ascii_lowercase + string.digits
password_length = 8
table_size = 1000000  # The size of the rainbow table

rainbow_table = create_rainbow_table()

hash_to_find = 'e06554818e902b4ba339f066967c0000da3fcda4fd7eb4ef89c124fa78bda419'

plaintext = search_rainbow_table(rainbow_table, hash_to_find)
if plaintext:
    print(f'Found plaintext for the hash: {plaintext}')
else:
    print('Plaintext not found in the rainbow table.')


Found plaintext for the hash: cryptography


# Dictionary attack

In [10]:
import hashlib

def dictionary_attack(target_hash, password_list):
    for password in password_list:
        hashed = hashlib.sha256(password.encode()).hexdigest()
        if hashed == target_hash:
            return password  # Password found
    return None  # Password not found

password_list = read_passwords_from_file('./10-million-password-list-top-1000000.txt')
target_hash = 'e06554818e902b4ba339f066967c0000da3fcda4fd7eb4ef89c124fa78bda419'
found_password = dictionary_attack(target_hash, password_list)
if found_password:
    print(f"Password found: {found_password}")
else:
    print("Password not found.")


Password not found.


# Hybrid attacks

In [5]:
import hashlib
import itertools
import string

def hybrid_attack(target_hash, base_password_list, max_extra_length=2):
    chars = string.ascii_lowercase + string.ascii_uppercase + string.digits + string.punctuation

    for base_password in base_password_list:
        for extra_length in range(1, max_extra_length + 1):
            for combination in itertools.product(chars, repeat=extra_length):
                test_password = base_password + ''.join(combination)
                hashed = hashlib.sha256(test_password.encode()).hexdigest()
                if hashed == target_hash:
                    return test_password
    return None

password_list = read_passwords_from_file('./10-million-password-list-top-1000000.txt')
target_hash = 'e06554818e902b4ba339f066967c0000da3fcda4fd7eb4ef89c124fa78bda419'
found_password = hybrid_attack(target_hash, password_list, max_extra_length=2)
if found_password:
    print(f"Password found: {found_password}")
else:
    print("Password not found.")


KeyboardInterrupt: 